In [5]:
import math
import pandas as pd
import geopandas as gpd
from shapely import wkt
from shapely.geometry import MultiLineString, Point

# ───── PARAMETERS ─────────────────────────────────────────────────────────────
CSV_PATH        = 'data_final.csv'  # adjust if needed
UTM_EPSG        = 32614                # metric CRS (UTM zone)
DIST_THRESHOLD  = 10                   # snapping radius in meters
ANGLE_THRESHOLD = math.radians(30)     # max allowed angle difference (radians)
# ────────────────────────────────────────────────────────────────────────────────

In [6]:
# 1) Load raw CSV
df = pd.read_csv(CSV_PATH)

# 2) Build gdf_links: one row per LINK_ID
df_links = df[['LINK_ID','MULTIDIGIT','geom_sn']].drop_duplicates().copy()
df_links['geometry'] = df_links['geom_sn'].apply(wkt.loads)
gdf_links = gpd.GeoDataFrame(
    df_links.drop(columns='geom_sn'),
    geometry='geometry',
    crs="EPSG:4326"
).to_crs(epsg=UTM_EPSG)

/var/folders/m2/3gj76pxj4137lqnxbvjtp2h40000gn/T/ipykernel_71629/73062908.py:2: DtypeWarning: Columns (8,9,26,31,34,36,37) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(CSV_PATH)


In [7]:
df

,LINK_ID,POI_ID,SEQ_NUM,FAC_TYPE,POI_NAME,POI_LANGCD,POI_NMTYPE,POI_ST_NUM,ST_NUM_FUL,ST_NFUL_LC,...,R_NREFADDR,R_REFADDR,ST_LANGCD_sn,ST_NAME_sn,ST_NM_BASE,ST_NM_SUFF,ST_TYP_AFT,ST_TYP_ATT,ST_TYP_BEF,geom_sn
0,851035131,1210140360,1,9992,IGLESIA PUERTA DE SALVACIÓN ALABANZA Y ADORACI...,SPA,B,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,N,NaN,"LINESTRING (-98.90419 19.38114, -98.90412 19.3..."
1,702764303,1059254882,1,8211,ESCUELA SECUNDARIA TÉCNICA INDUSTRIAL Y COMERC...,SPA,B,NaN,NaN,NaN,...,1.0,2.0,SPA,CALLE MARTÍN GARCÍA,MARTÍN GARCÍA,NaN,NaN,N,CALLE,"LINESTRING (-99.0122 19.60828, -99.01298 19.60..."
2,702706827,1244216018,1,9567,TIENDA ESCOLAR ESCUELA SECUNDARIA OF 323 JOSE ...,SPA,B,NaN,NaN,NaN,...,199.0,1.0,SPA,AVENIDA INDEPENDENCIA,INDEPENDENCIA,NaN,NaN,N,AVENIDA,"LINESTRING (-99.62585 19.3116, -99.6259 19.31291)"
3,1310846005,1206969826,1,9525,FISCALÍA DESCONCENTRADA DE INVESTIGACIÓN DE IZ...,SPA,B,NaN,NaN,NaN,...,30.0,30.0,SPA,PROLONGACIÓN TELECOMUNICACIONES,TELECOMUNICACIONES,NaN,NaN,N,PROLONGACIÓN,"LINESTRING (-99.04152 19.38159, -99.04188 19.3..."
4,703476040,1210138617,1,9992,TEMPLO EVANGÉLICO JESUS EN SAMARIA DE LAS ASAM...,SPA,B,NaN,NaN,NaN,...,4399.0,4301.0,SPA,CALLE NORTE 88,NORTE 88,NaN,NaN,N,CALLE,"LINESTRING (-99.10348 19.45269, -99.10304 19.4..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
186664,703604898,1200752158,1,9992,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,SPA,CALLE LACANDONES,LACANDONES,NaN,NaN,N,CALLE,"LINESTRING (-99.17665 19.27189, -99.1773 19.27..."
186665,702800883,1226322495,1,7520,NaN,NaN,NaN,NaN,NaN,NaN,...,410.0,404.0,SPA,CALLE MARIANO ESCOBEDO,MARIANO ESCOBEDO,NaN,NaN,N,CALLE,"LINESTRING (-99.19853 19.53977, -99.19971 19.5..."
186666,949507455,1163109904,1,7520,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,SPA,AVENIDA FRAY SERVANDO TERESA DE MIER,FRAY SERVANDO TERESA DE MIER,NaN,NaN,N,AVENIDA,"LINESTRING (-99.09945 19.41668, -99.09974 19.4..."
186667,703439151,1224497309,1,7520,NaN,NaN,NaN,NaN,NaN,NaN,...,105.0,101.0,SPA,CALLE BRAVO,BRAVO,NaN,NaN,N,CALLE,"LINESTRING (-99.1217 19.44017, -99.12162 19.4408)"


In [8]:
# 3) Compute bearing, D1, D2 on gdf_links
def compute_bearing(ls):
    if isinstance(ls, MultiLineString):
        ls = max(ls.geoms, key=lambda g: g.length)
    x0, y0 = ls.coords[0]
    x1, y1 = ls.coords[-1]
    return math.atan2(y1 - y0, x1 - x0)

def measure_separation(ls):
    if not isinstance(ls, MultiLineString) or len(ls.geoms) < 2:
        return pd.Series({'D1': None, 'D2': None})
    parts = sorted(ls.geoms, key=lambda g: g.length, reverse=True)[:2]
    D1 = parts[0].centroid.distance(parts[1].centroid)
    D2 = min(parts[0].length, parts[1].length)
    return pd.Series({'D1': D1, 'D2': D2})

gdf_links['orig_bearing'] = gdf_links.geometry.apply(compute_bearing)
sep_df = gdf_links.geometry.apply(measure_separation)
gdf_links = pd.concat([gdf_links, sep_df], axis=1)

# Ensure MULTIDIGIT column exists
if 'MULTIDIGIT' not in gdf_links.columns:
    flags = df[['LINK_ID','MULTIDIGIT']].drop_duplicates()
    gdf_links = gdf_links.merge(flags, on='LINK_ID', how='left')

In [9]:
# 4) Build gdf_pois by interpolating PERCFRREF along each link
df_pois = df[['LINK_ID','PERCFRREF','geom_sn']].copy()
df_pois['link_geom'] = df_pois['geom_sn'].apply(wkt.loads)
gdf_temp = gpd.GeoDataFrame(
    df_pois.drop(columns='geom_sn'),
    geometry=df_pois['link_geom'],
    crs="EPSG:4326"
).to_crs(epsg=UTM_EPSG)

gdf_temp['poi_geom'] = [
    line.interpolate(line.length * (pct / 100.0))
    for line, pct in zip(gdf_temp.geometry, gdf_temp.PERCFRREF)
]
gdf_pois = gpd.GeoDataFrame(
    gdf_temp[['LINK_ID']],
    geometry=gdf_temp['poi_geom'],
    crs=gdf_temp.crs
)

In [10]:
# 5) Spatial snap: nearest-link join within DIST_THRESHOLD
links_nn = (
    gdf_links
    .rename(columns={'LINK_ID':'nearest_LINK_ID'})
    [['nearest_LINK_ID','geometry','orig_bearing','MULTIDIGIT','D1','D2']]
)
joined = gpd.sjoin_nearest(
    gdf_pois,
    links_nn,
    how='left',
    max_distance=DIST_THRESHOLD,
    distance_col='dist_to_nearest'
)

In [11]:
# 6) Merge back POI’s original link bearing for angle comparison
joined = joined.merge(
    gdf_links[['LINK_ID','orig_bearing']].rename(columns={'orig_bearing':'poi_bearing'}),
    on='LINK_ID',
    how='left'
)

In [12]:
# 7) Compute minimal angle difference
def angle_diff(b1, b2):
    diff = abs(b1 - b2) % math.pi
    return diff if diff <= math.pi/2 else math.pi - diff

joined['angle_diff'] = joined.apply(
    lambda r: angle_diff(r['poi_bearing'], r['orig_bearing']),
    axis=1
)

In [13]:
# 8) Multi-digit flag consistency check
def multi_condition(row):
    flag = row['MULTIDIGIT']
    D1, D2 = row['D1'], row['D2']
    if flag == 'Y':
        return (D1 is not None and 3 < D1 <= 80 and D2 is not None and D2 > 40)
    else:
        return not (D1 is not None and 3 < D1 <= 80 and D2 is not None and D2 > 40)

joined['multi_ok'] = joined.apply(multi_condition, axis=1)

In [14]:
# 9) Final classification: valid only if all tests pass
joined['valid'] = (
    joined['dist_to_nearest'].notna() &
    (joined['dist_to_nearest'] <= DIST_THRESHOLD) &
    (joined['nearest_LINK_ID'] == joined['LINK_ID']) &
    (joined['angle_diff'] <= ANGLE_THRESHOLD) &
    joined['multi_ok']
)

In [24]:
# 10) Split into valid/invalid sets and save or inspect
valid_pois   = joined[joined['valid']].copy()
invalid_pois = joined[~joined['valid']].copy()

valid_pois.to_csv('valid_pois.csv', index=False)
invalid_pois.to_csv('invalid_pois.csv', index=False)
print(f"Valid POIs:   {len(valid_pois)}")
print(f"Invalid POIs: {len(invalid_pois)}")

Valid POIs:   159192
Invalid POIs: 28317
